In [4]:
pip install quandl

In [17]:
import quandl
import pandas as pd
import numpy as np
import pandas as pd
import math
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [9]:
df = quandl.get("WIKI/GOOGL")#Getting our data from quandl
df.head()

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2004-08-19,100.01,104.06,95.96,100.335,44659000.0,0.0,1.0,50.159839,52.191109,48.128568,50.322842,44659000.0
2004-08-20,101.01,109.08,100.50,108.310,22834300.0,0.0,1.0,50.661387,54.708881,50.405597,54.322689,22834300.0
2004-08-23,110.76,113.48,109.05,109.400,18256100.0,0.0,1.0,55.551482,56.915693,54.693835,54.869377,18256100.0
2004-08-24,111.24,111.60,103.57,104.870,15247300.0,0.0,1.0,55.792225,55.972783,51.945350,52.597363,15247300.0
2004-08-25,104.76,108.00,103.88,106.000,9188600.0,0.0,1.0,52.542193,54.167209,52.100830,53.164113,9188600.0


In [10]:
df = df[['Adj. Open',  'Adj. High',  'Adj. Low',  'Adj. Close', 'Adj. Volume']]#Taking only the data from dataframe that is useful for our operation. 
df['HL_PCT'] = (df['Adj. High'] - df['Adj. Low']) / df['Adj. Low'] * 100.0 #Adding an High - Low Percent to get volatility of our stock data.
df['PCT_change'] = (df['Adj. Close'] - df['Adj. Open']) / df['Adj. Open'] * 100.0 #Adding the percent change coloumn as well.
df = df[['Adj. Close', 'HL_PCT', 'PCT_change', 'Adj. Volume']]#Filtering our data to get rid of the redundant parts.

In [11]:
df.head()

,Adj. Close,HL_PCT,PCT_change,Adj. Volume
Date,,,,
2004-08-19,50.322842,8.441017,0.324968,44659000.0
2004-08-20,54.322689,8.537313,7.227007,22834300.0
2004-08-23,54.869377,4.062357,-1.227880,18256100.0
2004-08-24,52.597363,7.753210,-5.726357,15247300.0
2004-08-25,53.164113,3.966115,1.183658,9188600.0


In [18]:
forecast_col = 'Adj. Close' #since working on stock prices using adj close 
df.fillna(value=-99999, inplace=True)
forecast_out = int(math.ceil(0.01 * len(df)))

In [19]:
df['label'] = df[forecast_col].shift(-forecast_out)

In [20]:
df.head()

,Adj. Close,HL_PCT,PCT_change,Adj. Volume,label
Date,,,,,
2004-08-19,50.322842,8.441017,0.324968,44659000.0,69.078238
2004-08-20,54.322689,8.537313,7.227007,22834300.0,67.839414
2004-08-23,54.869377,4.062357,-1.227880,18256100.0,68.912727
2004-08-24,52.597363,7.753210,-5.726357,15247300.0,70.668146
2004-08-25,53.164113,3.966115,1.183658,9188600.0,71.219849


In [22]:
df

,Adj. Close,HL_PCT,PCT_change,Adj. Volume,label
Date,,,,,
2004-08-19,50.322842,8.441017,0.324968,44659000.0,69.078238
2004-08-20,54.322689,8.537313,7.227007,22834300.0,67.839414
2004-08-23,54.869377,4.062357,-1.227880,18256100.0,68.912727
2004-08-24,52.597363,7.753210,-5.726357,15247300.0,70.668146
2004-08-25,53.164113,3.966115,1.183658,9188600.0,71.219849
...,...,...,...,...,...
2018-03-21,1094.000000,1.976619,0.130884,1990515.0,NaN
2018-03-22,1053.150000,3.265882,-2.487014,3418154.0,NaN
2018-03-23,1026.550000,4.089299,-2.360729,2413517.0,NaN


In [23]:
df.dropna(inplace=True)

In [24]:
df

,Adj. Close,HL_PCT,PCT_change,Adj. Volume,label
Date,,,,,
2004-08-19,50.322842,8.441017,0.324968,44659000.0,69.078238
2004-08-20,54.322689,8.537313,7.227007,22834300.0,67.839414
2004-08-23,54.869377,4.062357,-1.227880,18256100.0,68.912727
2004-08-24,52.597363,7.753210,-5.726357,15247300.0,70.668146
2004-08-25,53.164113,3.966115,1.183658,9188600.0,71.219849
...,...,...,...,...,...
2018-01-30,1177.370000,1.142604,-0.029718,1792602.0,1094.000000
2018-01-31,1182.220000,1.213207,-0.134312,1643877.0,1053.150000
2018-02-01,1181.590000,1.547000,0.476195,2774967.0,1026.550000


In [25]:
#Initialising X and y with our feeatures and labels respectively.
X = np.array(df.drop(['label'],1))
y = np.array(df['label'])

In [26]:
X = preprocessing.scale(X)#Pre processing our data

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) #Splitting our data into training and testing data 0.2 is specified means 80% is our training data and the rest is testing

In [29]:
clf = svm.SVR()#Using the SupportVectorRegression

In [34]:
clf.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [35]:
confidence = clf.score(X_test,y_test)

In [32]:
confidence # with support vector regression we get an accuracy of ~80% which is okay lets try another clawssifier

0.7971488425153384

In [33]:
clf = LinearRegression()

In [36]:
confidence#with linear regression we get an 97% accuracy

0.9767354228649296

In [ ]:
#The thing is in SVR there is no threading there so it is less efficient compared to the linear regression algo used above.